# Entrega 2: Análisis Profundo según Enfoque Elegido
- A00836286 | Esteban Sierra Baccio
- A00837527 | Diego de Jesús Esparza Ruíz
- A01722667 | Javier Jorge Hernández Verduzco
- A01285193 | Sergio Omar Flores García
- A01613878 | Sergio Aarón Hernández Orta

## 1. Refinamiento y Operacionalización (1-2 páginas)

### 1.1 Ajustes a las preguntas de investigación basados en feedback de Entrega 1
### 1.2 Definición operacional precisa de cada variable y constructo
### 1.3 Justificación de cualquier cambio en el enfoque metodológico
### 1.4 Descripción detallada de cómo construyeron variables derivadas o índices
 

## 2. Implementación Metodológica Principal (5-7 páginas + código)

El contenido específico dependerá del enfoque elegido, pero TODOS deben incluir:

### Sección A: Preparación y Transformación

#### Proceso detallado de limpieza y transformación de datos
| Variable a eliminar | Razón |
| ---- | ---- |
| Viaje | Información irrelevante |
| Permiso | Información irrelevante |
| Shipment | Información irrelevante |
| Sociedad | Información irrelevante |
| Ori-Dest-TT | Información redundante |
| Ori-Dest | Información redundante |
| Modal | Información irrelevante |
| Moneda | Información irrelevante |
| Estatus | Información irrelevante |


In [1]:
# Leer los datos
import pandas as pd
import numpy as np

df = pd.read_excel("../data/raw/Viajes Sep-Dic 24 v2.xlsx", sheet_name='Viajes')

In [ ]:
# Limpieza de datos
df = df.drop(columns=['Viaje', 'Permiso', 'Shipment', 'Sociedad','Ori-Dest-TT','Ori-Dest','Modal.','Moneda','Estatus'])

# Se elimina la última linea que contiene totales
df = df.iloc[:-1]

df.head()


,Planta Origen,Tipo Planta,Origen,Destino,Deposito Origen,Tipo planta,Desc Org. Apt,TPP,Desc TPP,Transp.Leg,...,#Remitos,Peso Total (kg),Costo,Costo Prom,Variación,CostoxTn,Shp.Cost,Monto Real,Monto Falso,Monto Reparto
0,MVA,Revestidos,COA,AM Monterrey Local,MVAMI,Revestidos,Monclova,MVA,Planta Monclova,100128.0,...,1.0,28658.0,8000.00,8400.991615,-400.991615,279.154163,5.005105e+09,8000.00,0.0,0.0
1,CHU,Masivos,NL,Área Metro SLP,CHUCS,Customizados,Churubusco,CHU,Planta Churubusco,100139.0,...,1.0,8823.0,22956.30,23022.182000,-65.882000,2601.870112,5.005105e+09,22956.30,0.0,0.0
2,CHU,Masivos,NL,Área Metro SLP,CHUCS,Customizados,Churubusco,CHU,Planta Churubusco,100204.0,...,1.0,17646.0,25576.85,25108.522200,468.327800,1449.441800,5.005105e+09,25576.85,0.0,0.0
3,MVA,Revestidos,COA,Juarez,MVAMI,Revestidos,Monclova,MVA,Planta Monclova,100349.0,...,1.0,30354.0,40917.19,41019.101250,-101.911250,1347.999934,5.005105e+09,40917.19,0.0,0.0
4,UNI,Revestidos,NL,Gomez Palacio,UNIUN,Revestidos,Universidad,UNI,Planta Universidad,100204.0,...,1.0,28120.0,24850.00,22162.481520,2687.518480,883.712660,5.005105e+09,24850.00,0.0,0.0


### Feature: Distancia recorrida en KM

In [9]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

def calcular_distancia_recta(row):
    try:
        geolocator = Nominatim(user_agent="mi_app_geo") # Define tu user_agent
        
        # Concatena tus campos para darle más contexto a Nominatim
        query_origen = f"{row['Desc Org. Apt']}, {row['Origen']}"
        query_destino = f"{row['Destino']}"

        # Geocodifica el origen
        location_origen = geolocator.geocode(query_origen)
        time.sleep(1) # IMPORTANTE: Nominatim requiere un delay de 1 seg por petición
        
        # Geocodifica el destino
        location_destino = geolocator.geocode(query_destino)
        time.sleep(1)

        if location_origen and location_destino:
            coords_origen = (location_origen.latitude, location_origen.longitude)
            coords_destino = (location_destino.latitude, location_destino.longitude)
            
            # Calcula la distancia geodésica (línea recta)
            distancia_km = geodesic(coords_origen, coords_destino).kilometers
            print(f"Éxito: {query_origen} -> {query_destino} = {distancia_km:.2f} km")
            return distancia_km
        else:
            print(f"No se pudieron geocodificar: {query_origen} o {query_destino}")
            return np.nan

    except Exception as e:
        print(f"Error en Nominatim: {e}")
        return np.nan

In [ ]:
# Modificamos el DF para estandarizar nombres de Origen y Destino
df['Desc Org. Apt'] = df['Origen'].replace('Churubusco', 'Churubusco Monterrey')
df['Destino'] = df['Destino'].replace('AM Monterrey Plantas', 'Monterrey')
df['Destino'] = df['Destino'].replace('AM Monterrey Local', 'Monterrey')
df['Destino'] = df['Destino'].replace('Area Metr Queretaro', 'Queretaro')
df['Destino'] = df['Destino'].replace('Área Metro Saltillo', 'Saltillo')
df['Destino'] = df['Destino'].replace('Area Metro Monclova', 'Monclova')
df['Destino'] = df['Destino'].replace('Área Metro Monclova', 'Monclova')
df['Destino'] = df['Destino'].replace('Área Metro SLP', 'San Luis Potosi')
df['Destino'] = df['Destino'].replace('Área Metro Puebla', 'Puebla')
df['Destino'] = df['Destino'].replace('Área Metro Celaya', 'Celaya')
df['Destino'] = df['Destino'].replace('A. M. Aguascalientes', 'Aguascalientes')

# Creamos un dataframe con las combinaciones únicas de Origen y Destino
combinaciones = df[['Desc Org. Apt','Origen', 'Destino']].drop_duplicates()

# Obtenemos la distancia entre cada par de Origen y Destino
combinaciones['Distancia_km'] = combinaciones.apply(calcular_distancia_recta, axis=1)

# Añadimos la distancia al dataframe original
df = df.merge(combinaciones[['Desc Org. Apt','Origen', 'Destino', 'Distancia_km']], on=['Desc Org. Apt','Origen', 'Destino'], how='left')

# Guardamos las combinaciones en un archivo CSV
combinaciones.to_csv('../data/processed/combinaciones_origen_destino.csv', index=False)

#print(combinaciones)

Éxito: COA, COA -> Monterrey = 7898.17 km
Éxito: NL, NL -> San Luis Potosi = 3417.76 km
Éxito: COA, COA -> Juarez = 7740.31 km
Éxito: NL, NL -> Gomez Palacio = 3800.22 km
Éxito: COA, COA -> Monterrey = 7898.17 km
Éxito: NL, NL -> Saltillo = 3554.74 km
Éxito: NL, NL -> Monclova = 3626.49 km
Éxito: NL, NL -> Tepeji del Rio de O = 3241.15 km
Éxito: COA, COA -> Laredo, TX = 7689.39 km
Éxito: NL, NL -> Silao = 3475.94 km
Éxito: NL, NL -> Saltillo = 3554.74 km
Error en Nominatim: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Edo.+y+Cd.+de+M%C3%A9xico&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error en Nominatim: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Leon&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org


#### Justificación de decisiones sobre valores faltantes, outliers, y agregaciones
#### Creación de variables relevantes para su enfoque (ratios, interacciones, lags, etc.)
#### Validación de supuestos requeridos por sus métodos
 

⠀Sección B: Análisis Principal (varía según enfoque)

Si eligieron enfoque econométrico:

Especificación y estimación de modelos (efectos fijos, aleatorios, mixtos)
Tests de especificación (Hausman, Breusch-Pagan, etc.)
Análisis de elasticidades y efectos marginales
Descomposición de varianza (Oaxaca-Blinder, Shapley, etc.)
 

⠀Si eligieron optimización:

Formulación matemática del problema de optimización
Implementación y solución del modelo
Análisis de sensibilidad y precios sombra
Comparación entre solución óptima y realidad observada
Identificación de restricciones activas
 

⠀Si eligieron machine learning:

Feature engineering creativo y justificado
Comparación sistemática de algoritmos con justificación
Análisis de importancia de variables (SHAP, permutación, etc.)
Interpretación detallada del modelo seleccionado
Diagnóstico de sesgo-varianza
 

⠀Si eligieron análisis causal:

Construcción y justificación del DAG (Directed Acyclic Graph)
Identificación de confounders, mediadores, y colisionadores
Estrategia de identificación causal (IV, diferencias-en-diferencias, etc.)
Estimación de efectos causales con intervalos de confianza
Análisis de sensibilidad a violaciones de supuestos
 

⠀Si eligieron sistemas complejos:

Modelado de interacciones y retroalimentación
Identificación de puntos de inflexión o transiciones de fase
Análisis de propagación de efectos
Simulación de escenarios emergentes

## 3. Validación Rigurosa (2-3 páginas)

Validación cruzada temporal (no aleatoria - respeten la estructura temporal)
Tests de robustez: ¿Los resultados se mantienen bajo diferentes especificaciones?
Análisis de estabilidad temporal: ¿Las relaciones encontradas son consistentes?
Bootstrap o métodos de remuestreo para cuantificar incertidumbre
Diagnósticos específicos del método (multicolinealidad, heterocedasticidad, autocorrelación, etc.)
 

## 4. Análisis de Casos Especiales (2 páginas)

Investigación detallada de outliers o períodos anómalos
¿Qué revelan estos casos sobre su teoría?
¿Son errores, casos especiales, o evidencia contra sus hipótesis?
Análisis de subgrupos relevantes (por tipo de transportista, ruta, temporada, etc.)
 

## 5. Primeros Hallazgos y Respuestas Tentativas (3-4 páginas)

Para cada pregunta de investigación:

Evidencia empírica encontrada (con visualizaciones apropiadas)
Magnitud y significancia de los efectos (práctica, no solo estadística)
Interpretación en el contexto del negocio
Nivel de confianza en el hallazgo
Explicaciones alternativas que no pueden descartarse
 

⠀6. Reflexión Crítica sobre Limitaciones (1-2 páginas)

Supuestos que no pudieron validar completamente
Limitaciones de los datos para responder sus preguntas
Amenazas a la validez interna y externa
Sesgos potenciales en su análisis
Lo que su método NO puede revelar sobre el problema